In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-dataset/documentation.pdf
/kaggle/input/heart-disease-dataset/heart_statlog_cleveland_hungary_final.csv


In [2]:
df=pd.read_csv('/kaggle/input/heart-disease-dataset/heart_statlog_cleveland_hungary_final.csv')
df.head()


,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [3]:
#ensemble techniques
#randomforest
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  1190 non-null   int64  
 1   sex                  1190 non-null   int64  
 2   chest pain type      1190 non-null   int64  
 3   resting bp s         1190 non-null   int64  
 4   cholesterol          1190 non-null   int64  
 5   fasting blood sugar  1190 non-null   int64  
 6   resting ecg          1190 non-null   int64  
 7   max heart rate       1190 non-null   int64  
 8   exercise angina      1190 non-null   int64  
 9   oldpeak              1190 non-null   float64
 10  ST slope             1190 non-null   int64  
 11  target               1190 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 111.7 KB


In [4]:
cat_cols = ['chest pain type', 'sex','fasting blood sugar']
num_cols = [x for x in df.columns if x not in cat_cols+['target']]

# Now you can use cat_cols and num_cols as needed
num_cols

['age',
 'resting bp s',
 'cholesterol',
 'resting ecg',
 'max heart rate',
 'exercise angina',
 'oldpeak',
 'ST slope']

In [5]:
df.isna().sum()

age                    0
sex                    0
chest pain type        0
resting bp s           0
cholesterol            0
fasting blood sugar    0
resting ecg            0
max heart rate         0
exercise angina        0
oldpeak                0
ST slope               0
target                 0
dtype: int64

In [6]:
df.describe()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
count,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000
mean,53.720168,0.763866,3.232773,132.153782,210.363866,0.213445,0.698319,139.732773,0.387395,0.922773,1.624370,0.528571
std,9.358203,0.424884,0.935480,18.368823,101.420489,0.409912,0.870359,25.517636,0.487360,1.086337,0.610459,0.499393
min,28.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,0.000000,0.000000
25%,47.000000,1.000000,3.000000,120.000000,188.000000,0.000000,0.000000,121.000000,0.000000,0.000000,1.000000,0.000000
50%,54.000000,1.000000,4.000000,130.000000,229.000000,0.000000,0.000000,140.500000,0.000000,0.600000,2.000000,1.000000
75%,60.000000,1.000000,4.000000,140.000000,269.750000,0.000000,2.000000,160.000000,1.000000,1.600000,2.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,1.000000


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [8]:
xtr,xte,ytr,yte=train_test_split(df.drop('target',axis=1),df['target'], test_size=0.3)
for i in [xtr,xte,ytr,yte]:
    print(i.shape)

(833, 11)
(357, 11)
(833,)
(357,)


In [9]:
xtr.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope
234,41,1,4,112,250,0,0,142,0,0.0,1
449,63,1,4,160,230,1,0,105,1,1.0,2
185,54,0,2,160,312,0,0,130,0,0.0,1
219,55,1,2,140,196,0,0,150,0,0.0,1
269,54,1,4,130,242,0,0,91,1,1.0,2


In [10]:
df.columns

Index(['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol',
       'fasting blood sugar', 'resting ecg', 'max heart rate',
       'exercise angina', 'oldpeak', 'ST slope', 'target'],
      dtype='object')

In [11]:
# Scale numerical columns
mms = MinMaxScaler()
scaled_nums = mms.fit_transform(xtr[num_cols])
df_scaled_nums = pd.DataFrame(scaled_nums, columns=mms.get_feature_names_out(num_cols))

# Reset index of categorical columns DataFrame
xtr_cat_reset = xtr[cat_cols].reset_index(drop=True)  # Resetting index to avoid index mismatch

# Concatenate scaled numerical and categorical columns
xtr_scaled = pd.concat([df_scaled_nums, xtr_cat_reset], axis=1)
xtr_scaled.shape, ytr.shape

((833, 11), (833,))

In [12]:
temp_te=mms.transform(xte[num_cols])
df_scaled_nums=pd.DataFrame((temp_te),columns=mms.get_feature_names_out())

xte_scaled =pd.concat([df_scaled_nums, xte[cat_cols].reset_index(drop=True)] ,axis=1,ignore_index=True)
xte_scaled.shape,yte.shape

((357, 11), (357,))

In [13]:
score_base_models={}
ada=AdaBoostClassifier()
ada.fit(xtr,ytr)
score_base_models['AdaBoost'] =ada.score(xte,yte)
print(ada.score(xte,yte))
ypred=ada.predict(xte)
print(classification_report(ypred,yte))


0.8291316526610645
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       157
           1       0.86      0.83      0.85       200

    accuracy                           0.83       357
   macro avg       0.83      0.83      0.83       357
weighted avg       0.83      0.83      0.83       357



In [14]:
tuned={}
tuned['AdaBoost']=[]
params={"n_estimators":[10,20,50,100,150], "learning_rate":[0.1,0.2,0.3,0.5,0.8,1,2.0]}
gd=GridSearchCV(AdaBoostClassifier(),param_grid = params , n_jobs =-1, cv =5)
gd.fit(xtr,ytr)
tuned['AdaBoost'].append(gd.score(xte,yte))
tuned['AdaBoost'].append(gd.best_params_)
print(gd.score(xte,yte))
ypred=gd.predict(xte)
print(classification_report(ypred,yte))

0.8095238095238095
              precision    recall  f1-score   support

           0       0.76      0.81      0.78       152
           1       0.85      0.81      0.83       205

    accuracy                           0.81       357
   macro avg       0.81      0.81      0.81       357
weighted avg       0.81      0.81      0.81       357



In [15]:
# score_base_models={}
rf=RandomForestClassifier()
rf.fit(xtr,ytr)
score_base_models['RadomForest_base'] =rf.score(xte,yte)
print(rf.score(xte,yte))
ypred=rf.predict(xte)
print(classification_report(ypred,yte))

0.896358543417367
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       147
           1       0.94      0.88      0.91       210

    accuracy                           0.90       357
   macro avg       0.89      0.90      0.89       357
weighted avg       0.90      0.90      0.90       357



In [16]:
tuned['RadomForestClassifier']=[]
params={"n_estimators":[10,20,50,100,150], "criterion":['gini','entropy'], 'max_depth':[10,20,30,50],"bootstrap":[True,False]}
gd=GridSearchCV(RandomForestClassifier(),param_grid = params , n_jobs =-1, cv =5)
gd.fit(xtr,ytr)
tuned['RadomForestClassifier'].append(gd.score(xte,yte))
tuned['RadomForestClassifier'].append(gd.best_params_)
print(gd.score(xte,yte))
ypred=gd.predict(xte)
print(classification_report(ypred,yte))

0.8907563025210085
              precision    recall  f1-score   support

           0       0.83      0.92      0.87       147
           1       0.94      0.87      0.90       210

    accuracy                           0.89       357
   macro avg       0.89      0.89      0.89       357
weighted avg       0.90      0.89      0.89       357



In [17]:
# score_base_models={}
gbc=GradientBoostingClassifier()
gbc.fit(xtr,ytr)
score_base_models['GradientBoost'] =gbc.score(xte,yte)
print(gbc.score(xte,yte))
ypred=gbc.predict(xte)
print(classification_report(ypred,yte))

0.8739495798319328
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       155
           1       0.90      0.87      0.89       202

    accuracy                           0.87       357
   macro avg       0.87      0.87      0.87       357
weighted avg       0.88      0.87      0.87       357



In [18]:
tuned['GradientBoost']=[]
params={'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 1.0],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]}
gd=GridSearchCV(GradientBoostingClassifier(),param_grid = params , n_jobs =-1, cv =5)
history=gd.fit(xtr,ytr)
tuned['GradientBoost'].append(gd.score(xte,yte))
tuned['GradientBoost'].append(gd.best_params_)
print(gd.score(xte,yte))
ypred=gd.predict(xte)
print(classification_report(ypred,yte))

0.8935574229691877
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       158
           1       0.91      0.89      0.90       199

    accuracy                           0.89       357
   macro avg       0.89      0.89      0.89       357
weighted avg       0.89      0.89      0.89       357



In [19]:
tuned['GradientBoost']=[]
params={'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 1.0],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]}
gd=RandomizedSearchCV(GradientBoostingClassifier(),param_distributions = params , n_jobs =-1, cv =5)
history=gd.fit(xtr,ytr)
tuned['GradientBoost'].append(gd.score(xte,yte))
tuned['GradientBoost'].append(gd.best_params_)
print(gd.score(xte,yte))
ypred=gd.predict(xte)
print(classification_report(ypred,yte))

0.8851540616246498
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       157
           1       0.91      0.89      0.90       200

    accuracy                           0.89       357
   macro avg       0.88      0.89      0.88       357
weighted avg       0.89      0.89      0.89       357



In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
knn=KNeighborsClassifier(n_neighbors= 5)
knn.fit(xtr,ytr)
score_base_models['KNN'] =knn.score(xte,yte)
print(knn.score(xte,yte))
ypred=knn.predict(xte)
print(classification_report(ypred,yte))

0.7002801120448179
              precision    recall  f1-score   support

           0       0.70      0.66      0.68       171
           1       0.70      0.74      0.72       186

    accuracy                           0.70       357
   macro avg       0.70      0.70      0.70       357
weighted avg       0.70      0.70      0.70       357



In [21]:
tuned

{'AdaBoost': [0.8095238095238095, {'learning_rate': 0.2, 'n_estimators': 150}],
 'RadomForestClassifier': [0.8907563025210085,
  {'bootstrap': True,
   'criterion': 'gini',
   'max_depth': 50,
   'n_estimators': 100}],
 'GradientBoost': [0.8851540616246498,
  {'n_estimators': 50,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'max_depth': 5,
   'learning_rate': 0.1}]}

In [22]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
GNB=GaussianNB()
GNB.fit(xtr,ytr)
print(GNB.score(xte,yte))
ypred=GNB.predict(xte)
print(classification_report(ypred,yte))


0.8151260504201681
              precision    recall  f1-score   support

           0       0.79      0.80      0.80       160
           1       0.84      0.83      0.83       197

    accuracy                           0.82       357
   macro avg       0.81      0.81      0.81       357
weighted avg       0.82      0.82      0.82       357

